In [6]:
import streamlit as st

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

import matplotlib.ticker as ticker
import statsmodels.api as sm
import statsmodels.formula.api as smf

import plotly.express as px


from streamlit_extras.stylable_container import stylable_container
# from streamlit.extras import e
from eda import create_missing_values_barplot
from eda import create_damage_histogram
from eda import create_age_plotly
from eda import create_damage_below_15000_histogram
from eda import create_age_histogram
from eda import plot_categorical_barplots
from eda import create_age_plotly
from eda import create_damage_plotly

from modeling import prepare_data
from modeling import create_premium_df
from modeling import print_error_metrics

In [3]:
df = pd.read_excel('AutoBI.xlsx', sheet_name='Output')

In [7]:
# df.to_csv('AutoBI_output.csv', index=False)


In [9]:
categorical_cols = ['VEHICLE_TYPE', 'GENDER', 'MARITAL_STATUS', 'PREVCLM', 'SEATBELT']

feature_df = df.loc[:, ['INSAGE', 'VEHICLE_TYPE', 'GENDER', 'MARITAL_STATUS', 'PREVCLM', 'SEATBELT']]
target_series = df.loc[:, 'LOSS']


In [10]:
perc_testing = 30

In [23]:
train_feature_df, test_feature_df, train_target_series, test_target_series = train_test_split(
        feature_df, target_series, test_size=perc_testing/100, random_state=123)
    

In [24]:
len(test_target_series)

402

In [25]:
len(train_target_series)

938

In [27]:
predictors_to_remove = ['VEHICLE_TYPE', 'GENDER']

In [29]:
prepared_train_feature_df = prepare_data(train_feature_df)

In [30]:

model = sm.GLM(train_target_series, prepared_train_feature_df, family=sm.families.Gaussian()).fit()

In [32]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   LOSS   No. Observations:                  938
Model:                            GLM   Df Residuals:                      929
Model Family:                Gaussian   Df Model:                            8
Link Function:               Identity   Scale:                      1.4825e+09
Method:                          IRLS   Log-Likelihood:                -11230.
Date:                Wed, 20 Aug 2025   Deviance:                   1.3772e+12
Time:                        21:35:25   Pearson chi2:                 1.38e+12
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01559
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                              8354.9014   3412.415      2.448      0.014    1666.690     1.5e+04
VEHICLE_TYPE_Big car              -8560.9114   2543.520     -3.366      0.001   -1.35e+04   -3575.703
GENDER_Male                       -1810.2554   2570.357     -0.704      0.481   -6848.062    3227.551
MARITAL_STATUS_Divorced/Separated  3301.8192   8209.323      0.402      0.688   -1.28e+04    1.94e+04
MARITAL_STATUS_Married             4144.3483   2946.981      1.406      0.160   -1631.627    9920.324
MARITAL_STATUS_Widowed            -1905.8742   1.24e+04     -0.154      0.878   -2.62e+04    2.24e+04
PREVCLM_Yes                       -2433.7020   4413.893     -0.551      0.581   -1.11e+04    6217.370
SEATBELT_No                          40.7498   8726.389      0.005      0.996   -1.71e+04    1.71e+04
INSAGE                                5.1577     94.043      0.055      0.956    -179.163     189.478
=====================================================================================================
"""

In [40]:
prepared_feature_train_df = prepare_data(train_feature_df)
prepared_feature_test_df = prepare_data(test_feature_df)

In [41]:
create_premium_df(prepared_feature_test_df, test_target_series, model.predict(prepared_feature_test_df))

,index,const,VEHICLE_TYPE_Big car,GENDER_Male,MARITAL_STATUS_Divorced/Separated,MARITAL_STATUS_Married,MARITAL_STATUS_Widowed,PREVCLM_Yes,SEATBELT_No,INSAGE,target,expected_loss,diff,commission,reinsurance,admin,profit_margin,final_premium
0,642,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30.0,NaN,6699.377834,NaN,1339.875567,669.937783,669.937783,334.968892,9724.097859
1,524,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,NaN,6570.434643,NaN,1314.086929,657.043464,657.043464,328.521732,9537.130232
2,18,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,-2000.792190,NaN,-400.158438,-200.079219,-200.079219,-100.039609,-2891.148675
3,396,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,NaN,4216.855595,NaN,843.371119,421.685559,421.685559,210.842780,6124.440612
4,304,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,NaN,4031.177400,NaN,806.235480,403.117740,403.117740,201.558870,5855.207229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,628,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,48.0,NaN,4185.909229,NaN,837.181846,418.590923,418.590923,209.295461,6079.568382
398,385,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,32.0,NaN,-4284.920093,NaN,-856.984019,-428.492009,-428.492009,-214.246005,-6203.134135
399,200,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,27.0,NaN,4077.596948,NaN,815.519390,407.759695,407.759695,203.879847,5922.515575
400,1223,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,34.0,NaN,4113.701042,NaN,822.740208,411.370104,411.370104,205.685052,5974.866511


In [45]:
perc_testing

30

In [46]:
train_feature_df, test_feature_df, train_target_series, test_target_series = train_test_split(
        feature_df, target_series, test_size=perc_testing/100, random_state=1234)
    

In [47]:

prepared_feature_train_df.columns

Index(['const', 'VEHICLE_TYPE_Big car', 'GENDER_Male',
       'MARITAL_STATUS_Divorced/Separated', 'MARITAL_STATUS_Married',
       'MARITAL_STATUS_Widowed', 'PREVCLM_Yes', 'SEATBELT_No', 'INSAGE'],
      dtype='object')

In [49]:
prepared_feature_test_df.columns

Index(['const', 'VEHICLE_TYPE_Big car', 'GENDER_Male',
       'MARITAL_STATUS_Divorced/Separated', 'MARITAL_STATUS_Married',
       'MARITAL_STATUS_Widowed', 'PREVCLM_Yes', 'SEATBELT_No', 'INSAGE'],
      dtype='object')

In [60]:
prepared_train_feature_df

,const,VEHICLE_TYPE_Big car,GENDER_Male,MARITAL_STATUS_Divorced/Separated,MARITAL_STATUS_Married,MARITAL_STATUS_Widowed,PREVCLM_Yes,SEATBELT_No,INSAGE
1252,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
417,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,61.0
50,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,29.0
300,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,17.0
908,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...
1238,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0
1147,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,52.0
106,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,29.0
1041,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0


In [58]:
prepared_feature_train_df

,const,VEHICLE_TYPE_Big car,GENDER_Male,MARITAL_STATUS_Divorced/Separated,MARITAL_STATUS_Married,MARITAL_STATUS_Widowed,PREVCLM_Yes,SEATBELT_No,INSAGE
1252,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
417,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,61.0
50,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,29.0
300,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,17.0
908,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...
1238,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0
1147,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,52.0
106,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,29.0
1041,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0


In [61]:
train_target_series

1019     2275.0
1049        0.0
1144     2662.0
485      1198.0
378       635.0
         ...   
1228    11307.0
1077     1199.0
1318        0.0
723      1555.0
815      1050.0
Name: LOSS, Length: 938, dtype: float64

In [54]:
print(len(train_target_series))
print(prepared_feature_train_df.shape)

938
(938, 9)


In [51]:
def create_glm_model(target_series, prepared_feature_df, model_type='OLS'):
    if model_type == 'OLS':
        model = sm.OLS(target_series, prepared_feature_df).fit()
    elif model_type == 'Gamma':
        model = sm.GLM(target_series, prepared_feature_df, family=sm.families.Gamma(link=sm.families.links.Log())).fit()
    elif model_type == 'Tweedie':
        model = sm.GLM(target_series, prepared_feature_df, family=sm.families.Tweedie(var_power=1.5, link=sm.families.links.Log())).fit()
    return model
model_name = 'Gamma'
model = create_glm_model(train_target_series, prepared_feature_train_df, model_type=model_name)

ValueError: The indices for endog and exog are not aligned